# **FitBit Interactive Dashboard** #

The FitBit Activity Performance Dashboard shows the user it's progress over time

Four main perfomance indexes are displayed: 

    Mean steps
    Mean calories
    Mean activity (in terms of distance traveled)
    Mean sleep

Three main graphs are displayed:

    Calories over time
    Calories vs Totals (Steps, distance and sleep time)
    Activity minutes per category

**0) Data transforming and cleaning** 

In [141]:
#Importing libraries

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import matplotlib.pyplot as plt

Importing the databases

In [142]:
#Reading the main database that will be used (Activity Summary)

act_sum = pd.read_csv('dailyActivity_merged.csv')

#Reading the second database to complement the main database (Sleep data summary)

sleep_dat = pd.read_csv('sleepDay_merged.csv')

_The date column for both of the datasets is not in datetime format, so they will be transformed_ 

_Also, the column SleepDay will be change to ActivityDate to match the act_sum dataset for a futher join_

In [143]:
#Transforming the ActivityDate column to datetime format

act_sum['ActivityDate'] = pd.to_datetime(act_sum['ActivityDate'])

#Transforming the SleepDay column to datetime format

sleep_dat['SleepDay'] = pd.to_datetime(sleep_dat['SleepDay'])

#Renaming the SleepDay column to ActivityDate to match the main database

sleep_dat = sleep_dat.rename(columns={'SleepDay': 'ActivityDate'})

Performing an outer join using the columns ActivityDate and Id

In [144]:
#Merging the two datasets to have the data on only one dataset

tracker = pd.merge(act_sum, sleep_dat, on = ['ActivityDate', 'Id'], how = 'outer')

Cheking for nan values

In [145]:
tracker.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 943 entries, 0 to 942
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Id                        943 non-null    int64         
 1   ActivityDate              943 non-null    datetime64[ns]
 2   TotalSteps                943 non-null    int64         
 3   TotalDistance             943 non-null    float64       
 4   TrackerDistance           943 non-null    float64       
 5   LoggedActivitiesDistance  943 non-null    float64       
 6   VeryActiveDistance        943 non-null    float64       
 7   ModeratelyActiveDistance  943 non-null    float64       
 8   LightActiveDistance       943 non-null    float64       
 9   SedentaryActiveDistance   943 non-null    float64       
 10  VeryActiveMinutes         943 non-null    int64         
 11  FairlyActiveMinutes       943 non-null    int64         
 12  LightlyActiveMinutes  

_We can see there are rows with nan values_

_This is because some sleep data wasn't logged at the same date that the activity data was logged_

Dropping the rows with nan values and changing the Id type to str

In [146]:
#Dropping the nan values

tracker = tracker.dropna()

#Changing the Dtype of Id so it doesn't show as a integer in the visuals (not a quantity but a dimension)

tracker['Id'] = tracker['Id'].astype(str)

tracker.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413 entries, 0 to 905
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Id                        413 non-null    object        
 1   ActivityDate              413 non-null    datetime64[ns]
 2   TotalSteps                413 non-null    int64         
 3   TotalDistance             413 non-null    float64       
 4   TrackerDistance           413 non-null    float64       
 5   LoggedActivitiesDistance  413 non-null    float64       
 6   VeryActiveDistance        413 non-null    float64       
 7   ModeratelyActiveDistance  413 non-null    float64       
 8   LightActiveDistance       413 non-null    float64       
 9   SedentaryActiveDistance   413 non-null    float64       
 10  VeryActiveMinutes         413 non-null    int64         
 11  FairlyActiveMinutes       413 non-null    int64         
 12  LightlyActiveMinutes  

Creation of the TotalActiveMinutes column

In [147]:
tracker['TotalActiveMinutes'] = tracker['VeryActiveMinutes']+tracker['FairlyActiveMinutes']+tracker['LightlyActiveMinutes']+tracker['SedentaryMinutes']

tracker.head()

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,TotalActiveMinutes
0,1503960366,2016-04-12,13162,8.50,8.50,0.0,1.88,0.55,6.06,0.0,25,13,328,728,1985,1.0,327.0,346.0,1094
1,1503960366,2016-04-13,10735,6.97,6.97,0.0,1.57,0.69,4.71,0.0,21,19,217,776,1797,2.0,384.0,407.0,1033
3,1503960366,2016-04-15,9762,6.28,6.28,0.0,2.14,1.26,2.83,0.0,29,34,209,726,1745,1.0,412.0,442.0,998
4,1503960366,2016-04-16,12669,8.16,8.16,0.0,2.71,0.41,5.04,0.0,36,10,221,773,1863,2.0,340.0,367.0,1040
5,1503960366,2016-04-17,9705,6.48,6.48,0.0,3.19,0.78,2.51,0.0,38,20,164,539,1728,1.0,700.0,712.0,761


**1) Exploratory Analysis**

_**1.1) Mean performance indicators**_

_We will create a table with the mean perfomance indicators for each user_

In [148]:
#Creating the Id button to select different users

Id_button = pn.widgets.Select(name='Select User', options=tracker['Id'].unique().tolist())

Id_button

Select(options=['1503960366', ...], value='1503960366')

In [149]:
#Makign the dataframe interactive

idf = tracker.interactive()

In [150]:
#Pipeline to calculate the mean steps

meansteps_pl = (
    idf[
        idf.Id == Id_button]
    .groupby(['Id'])['TotalSteps'].mean().round()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

meansteps_pl

In [151]:
#Pipeline to calculate the mean distance

meandist_pl = (
    idf[
        idf.Id == Id_button]
    .groupby(['Id'])['TotalDistance'].mean().round()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

meandist_pl

In [152]:
#Pipeline to calculate the mean calories

meancal_pl = (
    idf[
        idf.Id == Id_button]
    .groupby(['Id'])['Calories'].mean().round()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

meancal_pl

In [153]:
#Pipeline to calculate the mean time asleep

meanasp_pl = (
    idf[
        idf.Id == Id_button]
    .groupby(['Id'])['TotalMinutesAsleep'].mean().round()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

meanasp_pl

In [154]:
#Tables showcasing the mean performance indicators

mean_clr = meancal_pl.pipe(pn.widgets.Tabulator, show_index=False, hidden_columns=['Id'], sizing_mode='stretch_both',
                          text_align='center', titles={'Calories': 'Mean Calories'}, header_align='center')

mean_stps = meansteps_pl.pipe(pn.widgets.Tabulator, show_index=False, hidden_columns=['Id'], sizing_mode='stretch_both',
                             text_align='center', titles={'TotalSteps': 'Mean Steps'}, header_align='center')

mean_dstnc = meandist_pl.pipe(pn.widgets.Tabulator, show_index=False, hidden_columns=['Id'], sizing_mode='stretch_both',
                             text_align='center', titles={'TotalDistance': 'Mean Distance'}, header_align='center')

mean_slp = meanasp_pl.pipe(pn.widgets.Tabulator, show_index=False, hidden_columns=['Id'], sizing_mode='stretch_both',
                          text_align='center', titles={'TotalMinutesAsleep': 'Mean Minutes Asleep'}, header_align='center')


_**1.2) Calories over time**_

_A plot of calories over time will be created for each user_

In [155]:
#Pipeline to get the calories over time

caltim_pl = (
    idf[
        idf.Id == Id_button]
    .groupby(['ActivityDate'])['Calories'].sum()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

caltim_pl

In [156]:
#Plot of calories over time

cal_plot = caltim_pl.hvplot(x='ActivityDate',y='Calories',title='Calories Over Time',
                               ylabel='Calories', xlabel='Date', width=1200)

cal_plot

_**1.3) Scatter plots**_

_Scatter plots will be created to showcase how the user's activity correlates to the calories burned_

In [157]:
#Creating the Id button to select different users

Cat_button = pn.widgets.Select(name='Select Category', options=['TotalSteps', 'TotalDistance','TotalMinutesAsleep'])

Cat_button

Select(options=['TotalSteps', ...], value='TotalSteps')

In [158]:
#Pipeline to get the calories for different categories

cat_scat_pl = (
    idf[
        idf.Id == Id_button]
    .groupby(Cat_button)['Calories'].sum()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

cat_scat_pl

In [159]:
#Plot of scatter plot

cat_plot = cat_scat_pl.hvplot.scatter(x='Calories', title='Activity Indicators vs Calories')

cat_plot

_**1.4) Stacked bar plot for activity**_

_A stacked bar plot will be created to showcase the different levels of activity_

In [160]:
#Pipeline to get the stacked bar of active minutes

actmin_pl = (
    idf[
        idf.Id == Id_button]
    .groupby(['ActivityDate', 'VeryActiveMinutes', 'FairlyActiveMinutes', 'LightlyActiveMinutes', 'SedentaryMinutes'])['TotalActiveMinutes'].mean()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

actmin_pl

In [161]:
#Plot for the stacked bar chart

activity_plot = actmin_pl.hvplot.bar(y=['VeryActiveMinutes', 'FairlyActiveMinutes', 'LightlyActiveMinutes', 'SedentaryMinutes'],
                                    stacked=True, ylabel='Minutes', xlabel='Records', legend='right', title='Active Minutes')

activity_plot

**2) Interactive dashboard**

In [162]:
#Layout using template
template = pn.template.FastListTemplate(
    title = 'FitBit Performance Indicators',
    sidebar=[pn.pane.Markdown('#  Improve Your Overall Health'),
            pn.pane.Markdown('Tracking your health with a Fitbit watch is important because it provides valuable insights into your daily activities, helps you make informed decisions about your health, and can motivate you to be more active and make healthier choices.'),
            pn.pane.PNG('Fitbit_image.png', sizing_mode='scale_both'),
            pn.pane.Markdown('## Settings'),
            Id_button, 
            pn.pane.Markdown(''),
            Cat_button],
    main = [pn.Row(
        pn.Column(mean_clr.panel(width=150), margin=(25)),
        pn.Column(mean_stps.panel(width=150), margin=(25)),
        pn.Column(mean_dstnc.panel(width=150), margin=(25)), 
        pn.Column(mean_slp.panel(width=150), margin=(25))),
            pn.Row(cal_plot.panel(width=900)),
            pn.Row(
        pn.Column(cat_plot.panel(width=900))),
            pn.Row(
        pn.Column(activity_plot.panel(width=900)))],
    accent_base_color = '#759EB8',
    header_background = '#759EB8',
    sidebar_width=320
)

#template.show()
#template.servable();